# Interactive 3D Time Series Visualization

This notebook demonstrates a visually stunning interactive visualization of median household income changes in Washington DC from 2012 to 2022 using:

- **pytidycensus**: Time series data with automatic boundary alignment
- **lonboard**: High-performance WebGL-based mapping
- **ipywidgets**: Interactive time slider

The visualization uses:
- Color coding for income levels
- 3D elevation proportional to income
- Interactive time slider to see changes over time
- Smooth transitions between years

## Installation

```bash
pip install pytidycensus[time] lonboard ipywidgets
```

In [26]:
# Import required libraries
import pytidycensus as tc
import pandas as pd
import geopandas as gpd
import numpy as np
from lonboard import Map, SolidPolygonLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import YlOrRd_9
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [27]:
# Set your Census API key
# Get a free key at: https://api.census.gov/data/key_signup.html

# tc.set_census_api_key("YOUR_API_KEY_HERE")

print("Remember to set your Census API key above!")

Remember to set your Census API key above!


## Step 1: Download Time Series Data

We'll use the new `get_time_series()` function to automatically handle boundary changes across years.

In [39]:
from pytidycensus.time_series import get_time_series

# Define years for time series (ACS 5-year data)
# Using 2015-2022 for reliable geometry downloads
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021]

print(f"Downloading median household income data for DC tracts...")
print(f"Years: {years}")
print(f"This will take a few minutes...\n")

# Download time series data with automatic boundary alignment
dc_income = tc.get_time_series(
    geography="tract",
    variables={"median_income": "B19013_001E"},
    years=years,
    dataset="acs5",
    state="DC",
    base_year=2021,  # Align all data to 2021 boundaries
    intensive_variables=["median_income"],  # Median is an intensive variable
    geometry=True,
    output="wide"
)

print(f"Downloaded data for {len(dc_income)} census tracts")
print(f"Data shape: {dc_income.shape}")
print(f"\nColumn structure (showing first few):")
print(dc_income.columns[:10].tolist())

Years: [2015, 2016, 2017, 2018, 2019, 2020, 2021]
This will take a few minutes...

Getting data from the 2011-2015 5-year ACS
Getting data from the 2012-2016 5-year ACS
Getting data from the 2013-2017 5-year ACS
Getting data from the 2014-2018 5-year ACS
Getting data from the 2015-2019 5-year ACS
Getting data from the 2016-2020 5-year ACS
Getting data from the 2017-2021 5-year ACS
Performing area interpolation for 2015 to 2021 boundaries...
Performing area interpolation for 2016 to 2021 boundaries...
Performing area interpolation for 2017 to 2021 boundaries...
Performing area interpolation for 2018 to 2021 boundaries...
Performing area interpolation for 2019 to 2021 boundaries...
Performing area interpolation for 2020 to 2021 boundaries...
DEBUG: Processing 7 years of data
  Year 2021: shape=(206, 8), columns=['GEOID', 'geometry', 'median_income', 'state', 'county', 'tract', 'NAME', 'B19013_001_moe']
  Year 2015: shape=(179, 8), columns=['GEOID', 'geometry', 'median_income', 'state', '

## Step 2: Prepare Data for Visualization

Transform the data to be compatible with Lonboard and prepare elevation scaling.

In [40]:
# Reproject to WGS84 (required by Lonboard)
dc_income = dc_income.to_crs("EPSG:4326")

# Extract income data for each year into separate columns
income_data = {}
for year in years:
    col_name = f"income_{year}"
    dc_income[col_name] = dc_income[(year, 'median_income')]
    income_data[year] = col_name
    
print("Income data prepared for visualization")
print(f"\nSummary statistics by year:")
for year in years:
    col = income_data[year]
    print(f"  {year}: ${dc_income[col].median():,.0f} (median), ${dc_income[col].mean():,.0f} (mean)")

Income data prepared for visualization

Summary statistics by year:
  2015: $71,354 (median), $75,466 (mean)
  2016: $75,289 (median), $78,360 (mean)
  2017: $79,440 (median), $82,936 (mean)
  2018: $84,375 (median), $87,428 (mean)
  2019: $94,004 (median), $92,417 (mean)
  2020: $98,654 (median), $100,792 (mean)
  2021: $102,414 (median), $105,142 (mean)


In [44]:
# Calculate global min/max for consistent color scaling across years
all_income_values = []
for year in years:
    col = income_data[year]
    all_income_values.extend(dc_income[col].dropna().tolist())

min_income = np.percentile(all_income_values, 5)  # Use 5th percentile to avoid outliers
max_income = np.percentile(all_income_values, 95)  # Use 95th percentile

print(f"\nIncome range for color scaling:")
print(f"  Min (5th percentile): ${min_income:,.0f}")
print(f"  Max (95th percentile): ${max_income:,.0f}")

# Define elevation scaling function
def calculate_elevation(income_values, min_val, max_val, max_height=4000):
    """
    Calculate elevation based on income values.
    Higher income = taller polygons.
    
    Parameters:
    -----------
    income_values : array-like
        Income values to scale
    min_val : float
        Minimum income for scaling
    max_val : float
        Maximum income for scaling
    max_height : float
        Maximum elevation in meters
    """
    # Normalize to 0-1 range
    normalized = (income_values - min_val) / (max_val - min_val)
    # Clip to ensure values are in [0, 1]
    normalized = np.clip(normalized, 0, 1)
    # Apply power transformation for better visual distinction
    normalized = np.power(normalized, 0.7)
    # Scale to maximum height
    return normalized * max_height

print(f"\nElevation scaling: 0 to 2000 meters based on income")

# Load water bodies to exclude from visualization
import os
water_bodies = None

# Try to load from local file first
water_file = None
for filename in ['waterbodies.geojson', 'Waterbodies.geojson']:
    potential_path = os.path.join('examples', 'data', filename)
    if os.path.exists(potential_path):
        water_file = potential_path
        break

if water_file:
    try:
        water_bodies = gpd.read_file(water_file)
        print(f"\n✓ Loaded water bodies from local file: {water_file}")
    except Exception as e:
        print(f"\n⚠ Error loading local water file: {e}")
        water_bodies = None

# If local file not found or failed, try downloading from DC GIS
if water_bodies is None:
    print("\n  Attempting to download water bodies from DC GIS...")
    dc_water_url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Environment_Water_WebMercator/MapServer/24/query?outFields=*&where=1%3D1&f=geojson'
    
    try:
        water_bodies = gpd.read_file(dc_water_url)
        print(f"✓ Successfully downloaded water bodies from DC GIS")
        
        # Optionally save for future use
        os.makedirs('examples/data', exist_ok=True)
        save_path = 'examples/data/Waterbodies.geojson'
        water_bodies.to_file(save_path, driver='GeoJSON')
        print(f"  Saved to {save_path} for future use")
    except Exception as e:
        print(f"⚠ Could not download water bodies: {e}")
        water_bodies = None

# Process water bodies if available
if water_bodies is not None:
    # Ensure same CRS
    water_bodies = water_bodies.to_crs(dc_income.crs)
    print(f"  Water bodies count: {len(water_bodies)}")
    

    print("  Calculating water overlap for each tract...")
    dc_income = dc_income.overlay(water_bodies, how='difference', keep_geom_type=True)
    dc_income.dissolve(by='GEOID', inplace=True)
    dc_income.reset_index(inplace=True)
    # remove small tract slivers created by overlay
    dc_income['area'] = dc_income.geometry.area
    area_threshold = dc_income['area'].median() * 0.35
    dc_income = dc_income[dc_income['area'] > area_threshold]
    dc_income = dc_income.drop(columns=['area'])    
 


Income range for color scaling:
  Min (5th percentile): $25,764
  Max (95th percentile): $176,906

Elevation scaling: 0 to 2000 meters based on income

✓ Loaded water bodies from local file: examples/data/Waterbodies.geojson
  Water bodies count: 1299
  Calculating water overlap for each tract...


KeyError: 'GEOID'

## Step 3: Create Interactive 3D Visualization

Build the Lonboard map with a time slider to explore income changes over time.

In [ ]:
def create_layer_for_year(year):
    """
    Create a SolidPolygonLayer for a specific year.
    Colors are mapped to the same values as elevation (income).
    """
    col = income_data[year]
    
    # Get income values
    income_values = dc_income[col].fillna(min_income).values
    
    # Calculate elevations based on income
    elevations = calculate_elevation(income_values, min_income, max_income)
    
    # IMPORTANT: Map colors to the SAME income values used for elevation
    # Normalize income to 0-1 range for color mapping
    income_normalized = (income_values - min_income) / (max_income - min_income)
    income_normalized = np.clip(income_normalized, 0, 1)
    
    # Apply colormap using the normalized income values
    # This ensures color directly corresponds to income/elevation
    colors = apply_continuous_cmap(
        income_normalized,  # Use normalized values, not raw income
        YlOrRd_9,
        alpha=0.8
    )
    
    # Create a simple GeoDataFrame with just geometry
    # Extract geometry column (it's not in the multi-index)
    gdf_clean = gpd.GeoDataFrame(geometry=dc_income.geometry, crs=dc_income.crs)
    
    # Create layer with explicit parameters
    layer = SolidPolygonLayer.from_geopandas(
        gdf_clean,
        get_elevation=elevations,
        get_fill_color=colors,
        extruded=True,
        wireframe=True,
        pickable=True,
        auto_downcast=False  # Disable auto downcasting to avoid pandas/arrow issues
    )
    
    return layer

# Create initial layer for first year
initial_layer = create_layer_for_year(years[0])

# Create map centered on DC
m = Map(
    layers=[initial_layer],
    view_state={
        'latitude': 38.9072,
        'longitude': -77.0369,
        'zoom': 11,
        'pitch': 45,
        'bearing': 0
    },
    height=700
)

print("Interactive map created!")
print("\nVisualization features:")
print("  - Color: Yellow (low income) to Red (high income)")
print("  - Height: Proportional to median household income")
print("  - Color and height are synchronized to income values")
print("  - Use slider below to explore different years")
print("  - Click and drag to rotate the view")
print("  - Scroll to zoom in/out")

Interactive map created!

Visualization features:
  - Color: Yellow (low income) to Red (high income)
  - Height: Proportional to median household income
  - Color and height are synchronized to income values
  - Use slider below to explore different years
  - Click and drag to rotate the view
  - Scroll to zoom in/out


In [ ]:
# Create year slider
year_slider = widgets.IntSlider(
    value=years[0],
    min=years[0],
    max=years[-1],
    step=3,  # 3-year increments
    description='Year:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='600px')
)

# Create info label
info_label = widgets.HTML(
    value=f"<b>Median Household Income - Washington DC Census Tracts</b><br>"
          f"<i>Data from ACS 5-year estimates, aligned to 2022 boundaries</i><br>"
          f"Year: {years[0]}"
)

# Create statistics display
stats_label = widgets.HTML()

def update_stats(year):
    col = income_data[year]
    median_val = dc_income[col].median()
    mean_val = dc_income[col].mean()
    min_val = dc_income[col].min()
    max_val = dc_income[col].max()
    
    stats_label.value = f"""
    <div style='background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-top: 10px;'>
        <b>Statistics for {year}:</b><br>
        Median: ${median_val:,.0f}<br>
        Mean: ${mean_val:,.0f}<br>
        Range: ${min_val:,.0f} - ${max_val:,.0f}
    </div>
    """

# Update function for slider
def update_map(change):
    year = change['new']
    
    # Update layer
    new_layer = create_layer_for_year(year)
    m.layers = [new_layer]
    
    # Update info
    info_label.value = f"<b>Median Household Income - Washington DC Census Tracts</b><br>" \
                      f"<i>Data from ACS 5-year estimates, aligned to 2022 boundaries</i><br>" \
                      f"Year: {year}"
    
    # Update statistics
    update_stats(year)

# Initialize stats
update_stats(years[0])

# Connect slider to update function
year_slider.observe(update_map, names='value')

# Create layout
controls = widgets.VBox([
    info_label,
    year_slider,
    stats_label
])

# Display everything
display(controls)
display(m)

Map(custom_attribution='', height='700px', layers=(SolidPolygonLayer(extruded=True, get_elevation=arro3.core.C…